In [ ]:
import cv2
import numpy as np
import skimage
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def show_image(image, cmap = None, fig_size = (10, 10)):
    fig, ax = plt.subplots(figsize=fig_size)
    ax.imshow(image, cmap = cmap)
    ax.axis('off')
    plt.show()

In [ ]:
image = cv2.imread('../img/seal.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

In [ ]:
show_image(np.flip(image, axis = 2))

## 1. Thresholding

### 1.1 Simple Thresholding

In [ ]:
(T, threshInv) = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY_INV)
# Threshold Binary Inverse
show_image(threshInv, cmap = 'gray')

In [ ]:
# using normal thresholding (rather than inverse thresholding),
# we can change the last argument in the function to make the coins
# black rather than white.
(T, thresh) = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
# Threshold Binary
show_image(thresh, cmap = 'gray')

In [ ]:
# finally, we can visualize only the masked regions in the image
bitwise_output = cv2.bitwise_and(image, image, mask=threshInv)
show_image(np.flip(image, axis = 2))
show_image(np.flip(bitwise_output, axis = 2))

### 1.2 Otsu's Method

In [ ]:
# apply Otsu's automatic thresholding -- Otsu's method automatically
# determines the best threshold value `T` for us
(T, threshInv) = cv2.threshold(blurred, 0, 255,
    cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
print("Otsu's thresholding value: {}".format(T))
show_image(threshInv, cmap = 'gray') # same as cv2.imshow("Threshold", threshInv)
 
# finally, we can visualize only the masked regions in the image
bitwise_and = cv2.bitwise_and(image, image, mask=threshInv)
show_image(bitwise_and, cmap = 'gray') # same as cv2.imshow("Output", bitwise_and)

### 1.3 Adaptive Thresholding

In [ ]:
# instead of manually specifying the threshold value, we can use adaptive
# thresholding to examine neighborhoods of pixels and adaptively threshold
# each neighborhood -- in this example, we'll calculate the mean value
# of the neighborhood area of 25 pixels and threshold based on that value;
# finally, our constant C is subtracted from the mean calculation (in this
# case 15)
thresh = cv2.adaptiveThreshold(blurred, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 35, 5)
show_image(thresh, cmap = 'gray') # same as cv2.imshow("OpenCV Mean Thresh", thresh)
 
# personally, I prefer the scikit-image adaptive thresholding, it just
# feels a lot more "Pythonic"
T = skimage.filters.threshold_local(blurred, 33, offset=4, method="gaussian")
thresh = (blurred < T).astype("uint8") * 255
show_image(thresh, cmap = 'gray') # cv2.imshow("scikit-image Mean Thresh", thresh)

### EXERCISE: Apply all three types of thresholding to another image - Which one has performmed better? Why?

In [ ]:
# TODO: Apply the above three thresholding methods to another image. Which has performed better? - You may need to fine tune Hyperparameters

## 2. Image Gradients

### 2.1 Sobel and Scharr Derivatives

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('../img/bricks.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# compute gradients along the X and Y axis, respectively
sobelx = cv2.Sobel(gray, ddepth = cv2.CV_64F, dx=1, dy=0)
sobely = cv2.Sobel(gray, ddepth = cv2.CV_64F, dx=0, dy=1)

# the gx and gy images are now of the floating point data type
# so we need to take care to convert them back to an unsigned 8-bit
# integer representation so other opencv functions can utilise them
sobelx = cv2.convertScaleAbs(sobelx)
sobely = cv2.convertScaleAbs(sobely)
sobel_combined = cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)

# Compute combined gradients using Laplacian Algorithm (and convert floating values to ints)
laplacian = cv2.Laplacian(gray,cv2.CV_64F)
laplacian = cv2.convertScaleAbs(laplacian)

plt.figure(figsize=(20,10))
plt.subplot(2,3,1),plt.imshow(gray,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(2,3,2),plt.imshow(sobelx,cmap = 'gray')
plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
plt.subplot(2,3,3),plt.imshow(sobely,cmap = 'gray')
plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
plt.subplot(2,3,4),plt.imshow(sobel_combined,cmap = 'gray')
plt.title('Sobel Combined'), plt.xticks([]), plt.yticks([])
plt.subplot(2,3,5),plt.imshow(laplacian,cmap = 'gray')
plt.title('Laplacian'), plt.xticks([]), plt.yticks([])

plt.show()

### 2.2 Output Datatypes

In [ ]:
img = cv2.imread('../img/bricks.png',0)[0:200, 0:200]

# Output dtype = cv2.CV_8U
sobelx8u = cv2.Sobel(img,cv2.CV_8U,1,0)

# Output dtype = cv2.CV_64F. Then take its absolute and convert to cv2.CV_8U
sobelx64f = cv2.Sobel(img,cv2.CV_64F,1,0)
abs_sobel64f = np.absolute(sobelx64f)
sobel_8u = np.uint8(abs_sobel64f)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1),plt.imshow(img,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(1,3,2),plt.imshow(sobelx8u,cmap = 'gray')
plt.title('Sobel CV_8U'), plt.xticks([]), plt.yticks([])
plt.subplot(1,3,3),plt.imshow(sobel_8u,cmap = 'gray')
plt.title('Sobel abs(CV_64F)'), plt.xticks([]), plt.yticks([])

plt.show()

### EXERCISE: Change kernel sizes in the above Sobel functions. What happens? Why?

In [ ]:
# TODO: Your code below

### 2.3 Gradient Orientation and Magnitude

In [ ]:
img = cv2.imread('../img/bricks.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# compute gradients along the X and Y axis, respectively
gX = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
gY = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
 
# compute the gradient magnitude and orientation respectively
mag = np.sqrt((gX ** 2) + (gY ** 2))
orientation = np.arctan2(gY, gX) * (180 / np.pi) % 180

# Printing magnitude and orientation at pixel (100, 100)
print('Magnitude at pixel (100, 100):', mag[100, 100])
print('Orientation at pixel (100, 100):', orientation[100, 100])

### 2.4 Detecting Gradient Changes using Orientation

In [ ]:
lower_angle = 90
upper_angle = 100

# find all pixels that are within the upper and low angle boundaries
idxs = np.where(orientation >= lower_angle, orientation, -1)
idxs = np.where(orientation <= upper_angle, idxs, -1)
mask = np.zeros(gray.shape, dtype="uint8")
mask[idxs > -1] = 255
 
# show the images
plt.figure(figsize=(20,10))
plt.subplot(1,2,1),plt.imshow(gray,cmap = 'gray')
plt.title('Original'), plt.xticks([]), plt.yticks([])
plt.subplot(1,2,2),plt.imshow(mask,cmap = 'gray')
plt.title('Mask'), plt.xticks([]), plt.yticks([])
plt.show()

### EXERCISE: Calculating Orientation and Magnitude

In [ ]:
# TODO: Load the seal image and calculate the gradient orientation and magnitude of the image at pixel 100, 100

## 3. Edge Detection

### 3.1 Canny Edge Detection

In [ ]:
image = cv2.imread("../img/bricks.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 10, 250)

plt.figure(figsize=(20,10))
plt.subplot(121),plt.imshow(gray,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Canny Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

### 3.2 Tuning Edge Detection Parameters

In [ ]:
# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread('../img/bricks.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)

# compute a "wide", "mid-range", and "tight" threshold for the edges
wide = cv2.Canny(blurred, 10, 200)
mid = cv2.Canny(blurred, 30, 150)
tight = cv2.Canny(blurred, 240, 250)
 
# show the edge maps
plt.figure(figsize=(20,10))
plt.subplot(221),plt.imshow(blurred,cmap = 'gray')
plt.title('Blurred Image'), plt.xticks([]), plt.yticks([])
plt.subplot(222),plt.imshow(wide,cmap = 'gray')
plt.title('Wide Edge Map'), plt.xticks([]), plt.yticks([])
plt.subplot(223),plt.imshow(mid,cmap = 'gray')
plt.title('Mid Edge Map'), plt.xticks([]), plt.yticks([])
plt.subplot(224),plt.imshow(tight,cmap = 'gray')
plt.title('Tight Edge Map'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
 
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
 
    # return the edged image
    return edged

In [ ]:
auto = auto_canny(blurred)

In [ ]:
# show the image
show_image(auto, cmap = 'gray')

### EXERCISE: Apply both kernels to an image - Which one has performmed better? Why?